### Primary Key del file `.FAB`

Ogni **record** del file `.FAB` è identificato in modo univoco dalla combinazione dei seguenti **6 campi**:

| Campo | Nome esteso | Descrizione |
|:------|:-------------|:------------|
| **CODAMM** | Codice Amministrativo | Identifica il Comune catastale di riferimento. |
| **SEZ** | Sezione | Campo attualmente non utilizzato (vuoto). |
| **IDIMMO** | Identificativo Immobile | Progressivo univoco dell’immobile (Unità Immobiliare Urbana) nella banca dati. |
| **TIPOIMMO** | Tipo Immobile | Assume valore fisso **F**, che indica *fabbricato* (Catasto Fabbricati). |
| **PROGRES** | Progressivo della Situazione | Numero progressivo che individua lo *stato oggettivo* dell’immobile nel tempo (versione o mutazione). |
| **TIPOREC** | Tipo di Record | Identifica la natura dei dati contenuti nella riga (1–6). |

In sintesi: la combinazione **(CODAMM, SEZ, IDIMMO, TIPOIMMO, PROGRES, TIPOREC)** consente di distinguere in modo univoco ogni record all’interno del file `.FAB`.

---

### Tipologie di record (`TIPOREC`)

Ogni **Unità Immobiliare Urbana (UIU)** è descritta da uno o più record, con **TIPOREC** variabile da 1 a 6.  
Ogni tipo di record contiene informazioni diverse, ma tutti condividono la stessa PK.

| `TIPOREC` | Descrizione | Contenuto principale |
|:-----------:|:-------------|:----------------------|
| **1** | Dati oggettivi dell’unità immobiliare | Categoria catastale, classe, consistenza, superficie, rendita, dati di atto, piani, partita catastale. |
| **2** | Identificativi catastali | Comune catastale (CC), foglio, particella, subalterno, porzioni materiali. |
| **3** | Indirizzo e ubicazione | Toponimo/località, indirizzo in italiano e tedesco, numeri civici. |
| **4** | Utilità comuni | Elenco dei beni comuni non censibili (es. scale, cortili, aree condominiali). |
| **5** | Riserve catastali | Segnalazioni tecniche e codici di riserva relativi all’immobile. |
| **6** | Annotazioni | Note testuali, motivazioni o respingimenti legati alle operazioni catastali. |

---

### Note

- I record con stesso **CODAMM + IDIMMO + PROGRES** appartengono **alla stessa UIU** (stesso immobile e stato).  
- I record di tipo 2–6 sono **record aggiuntivi** legati al record principale di tipo 1.  
- Per costruire un dataset “flat” (una riga per immobile), è necessario **aggregare** o **collegare** i record con la stessa chiave `(CODAMM, IDIMMO, PROGRES)`.

*Ancora una volta: la combinazione dei 6 campi che abbiamo detto all'inizio identifica univocamente un record del file FAB. la combinazione dei 3 campi di cui abbiamo parlato alla fine invece ci permette di identificare univocamente un immobile in un certo stato.*


In [11]:
import pandas as pd
from pathlib import Path

In [12]:
from pathlib import Path


# Il percorso è relativo alla posizione del notebook
fab_file = "../../../dati_AI_per_Challenge/catasto_fabbricati/IDR0000192470_TIPOFACSN_CAMML378.FAB"
"""
Handle the IndexError and the empty strings.
Note that not all of the rows in the FAB file have the same length (some records are shorter than others)
"""
def safe(parts, i):
    return parts[i].strip() if i < len(parts) and parts[i].strip() != "" else None

"""
Dict that contains all the different types we can find in this file.
ex. fab_records['1'] will contain all type 1 records.
"""
fab_records = {str(i): [] for i in range(1, 7)}  # types 1–6

with open(fab_file, encoding="utf-8", errors="ignore") as f:
    for line in f:
        parts = line.strip().split("|")
        if len(parts) < 6:
            #print(f'we have an invalid row: {parts}') # -> perfect: no invalid rows
            continue
        CODAMM = safe(parts, 0)
        SEZ = safe(parts, 1)
        IDIMMO = safe(parts, 2)
        TIPOIMMO = safe(parts, 3)
        PROGRES = safe(parts, 4)
        TIPOREC = safe(parts, 5)

        key = (CODAMM, IDIMMO, PROGRES) # this is unique for a property in a certain condition.
        base = {
            "CODAMM": CODAMM,
            "SEZ": SEZ,
            "IDIMMO": IDIMMO,
            "TIPOIMMO": TIPOIMMO,
            "PROGRES": PROGRES,
            "TIPOREC": TIPOREC,
        }
        """
        As we previously stressed: we have 6 different record types.
        """
        # Record type 1
        if TIPOREC == "1":
            fab_records["1"].append({
                **base,
                "ZONA": safe(parts, 6),
                "CATEGORIA": safe(parts, 7),
                "CLASSE": safe(parts, 8),
                "CONSISTENZA": safe(parts, 9),
                "SUPERFICIE": safe(parts, 10),
                "RENDITA_EURO": safe(parts, 13),
                "VALIMIS": safe(parts, 14),
                "PIANI": "|".join([p for p in parts[18:22] if p]),
                "DATAEFFINI": safe(parts, 29),
                "DATAEFFFIN": safe(parts, 30),
                "TIPONOTAINI": safe(parts, 31),
                "NUMNOTAINI": safe(parts, 32),
                "ANNO_NOTA": safe(parts, 34),
                "PARTITAIMM": safe(parts, 42),
                "IDMUTFIN": safe(parts, 44),
            })

        # Record type 2 (identificativi)
        elif TIPOREC == "2":
            fab_records["2"].append({
                **base,
                "COMUNE_CATASTALE": safe(parts, 6),
                "FOGLIO": safe(parts, 7),
                "PARTICELLA": safe(parts, 8),
                "SUBALTERNO": safe(parts, 9),
            })

        # Record type 3 (indirizzi)
        elif TIPOREC == "3":
            fab_records["3"].append({
                **base,
                "TOPONIMO": safe(parts, 6),
                "INDIRIZZO_ITA": safe(parts, 7),
                "CIVICO": safe(parts, 9),
            })

        # Record type 4 (utilità comuni)
        elif TIPOREC == "4":
            fab_records["4"].append({
                **base,
                "UCOM_CCPART": safe(parts, 6),
                "UCOM_FOGLIO": safe(parts, 7),
                "UCOM_NUMPART": safe(parts, 8),
                "UCOM_SUB": safe(parts, 9),
            })

        # Record type 5 (riserve)
        elif TIPOREC == "5":
            fab_records["5"].append({
                **base,
                "RISERVA_COD": safe(parts, 6),
                "RISERVA_DESCR": safe(parts, 7),
            })

        # Record type 6 (annotazioni)
        elif TIPOREC == "6":
            fab_records["6"].append({
                **base,
                "COD_ANN": safe(parts, 6),
                "TESTOANN": safe(parts, 7),
            })

# convert to dataframe each fab_records[x] list.
dfs = {k: pd.DataFrame(v) for k, v in fab_records.items() if len(v) > 0}
for k, df in dfs.items():
    print(f"Type {k}: {len(df)} record")

# create the flat table
fab1 = dfs.get("1", pd.DataFrame())
fab2 = dfs.get("2", pd.DataFrame())
fab3 = dfs.get("3", pd.DataFrame())

# aggregate the dataframes
def agg_text(df, col, keycols=["CODAMM", "IDIMMO", "PROGRES"]):
    return (
        df.groupby(keycols)[col]
        .apply(lambda x: "; ".join([str(i) for i in x if i]))
        .reset_index()
    )

fab2agg = agg_text(fab2, "PARTICELLA")
fab3agg = agg_text(fab3, "INDIRIZZO_ITA")
fab4agg = agg_text(fab2, "COMUNE_CATASTALE")

flat = fab1.merge(fab2agg, on=["CODAMM", "IDIMMO", "PROGRES"], how="left") \
           .merge(fab3agg, on=["CODAMM", "IDIMMO", "PROGRES"], how="left") \
           .merge(fab4agg, on=["CODAMM", "IDIMMO", "PROGRES"], how="left")

# clean the dataframe
flat["RENDITA_EURO"] = (
    flat["RENDITA_EURO"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.extract(r"([\d.]+)", expand=False)
    .astype(float)
)

flat = flat.sort_values(["CODAMM", "IDIMMO", "PROGRES"])
print("Total UIU:", len(flat))


null_cols = flat.columns[flat.isna().all()].tolist()
print("completely empty column:", null_cols)
flat = flat.drop(columns=null_cols)

flat.head(20)


Type 1: 192521 record
Type 2: 193672 record
Type 3: 148080 record
Type 4: 6691 record
Type 5: 28 record
Type 6: 1811 record
Total UIU: 192521
completely empty column: ['SEZ', 'PARTITAIMM']


,CODAMM,IDIMMO,TIPOIMMO,PROGRES,TIPOREC,ZONA,CATEGORIA,CLASSE,CONSISTENZA,SUPERFICIE,...,PIANI,DATAEFFINI,DATAEFFFIN,TIPONOTAINI,NUMNOTAINI,ANNO_NOTA,IDMUTFIN,PARTICELLA,INDIRIZZO_ITA,COMUNE_CATASTALE
0,L378,1,F,3,1,003,A02,03,10,275,...,T|1|2,None,None,14092006,03112006,007454,837238,00097,VILLA - BASELGA DEL BONDONE,017
9,L378,10,F,1,1,003,A03,01,"4,5",103,...,T|2|3,None,None,31101991,14071993,004216,69197,00114,OMALGA - BASELGA DEL BONDONE,017
99,L378,100,F,3,1,003,A04,01,4,78,...,T|2|3,None,None,31071992,26041994,004332,132752,00080,VILLA - BASELGA DEL BONDONE,017
999,L378,1000,F,2,1,002,C06,04,55,61,...,S1,None,None,25052012,31072012,007183,1156536,00182,DELLA CASARA - CADINE,050
9996,L378,10000,F,2,1,002,C06,03,18,20,...,S1,None,None,13072000,17102000,004361,545503,01745,ALLA PELEGRINA - COGNOLA,108
99961,L378,100000,F,3,1,001,C06,02,14,16,...,S2,None,None,04082000,25102000,004606,546584,06460,R. LUNELLI,406
99962,L378,100001,F,2,1,001,C06,02,13,14,...,S2,None,None,13062000,05092000,003668,541010,06460,R. LUNELLI,406
99963,L378,100002,F,2,1,001,C06,02,14,16,...,S2,None,None,22102001,22102001,004344,582264,06460,R. LUNELLI,406
99964,L378,100003,F,1,1,001,C06,02,14,16,...,S2,None,None,05101998,19111998,D00014,458952,06460,R. LUNELLI,406
156806,L378,1000036,F,1,1,002,D10,None,None,None,...,S1|T,None,None,10052012,13062012,005922,1150035,01715,PONTE ALTO - COGNOLA,108


#Docfa XML files

# Procedura DOCFA — Struttura e Significato

## Cos’è DOCFA

**DOCFa** serve per comunicare al **Catasto**:

- la **creazione di nuovi immobili** (nuove costruzioni)  
- le **modifiche a immobili esistenti** (frazionamenti, fusioni, ampliamenti, cambi d’uso, ecc.)  
- oppure **situazioni particolari** (immobile distrutto, in costruzione, ecc.)

---

## Struttura di un file Docfa

Un **singolo file Docfa** può contenere **più immobili o più particelle**.

### Intestazioni possibili

- **Diverse** → se si tratta di **nuove costruzioni**  
  **Identificato dal tag `<Accatastamento>`**

- **Uniche** → se si tratta di **variazioni** (es. fusione di unità di uno stesso proprietario)  
  **Identificato dal tag `<Variazione>`**

**In sintesi:**

| Tipo di file | Proprietari | Tag XML |
|---------------|--------------|----------|
| Nuova costruzione | Possono essere diversi | `<Accatastamento>` |
| Variazione | Devono essere gli stessi | `<Variazione>` |

Quindi, più immobili nello stesso file:  
se è **una variazione**, stesso proprietario;  
se è **una nuova costruzione**, proprietari diversi.

---

## Componenti di un documento Docfa

Ogni Docfa è composto da **modelli standard**:

- **Modello D1** → dati generali della dichiarazione (chi presenta, perché, dove, ecc.)  
- **Modello 1N – Parte I e II** → per unità delle categorie **A, B, C**  
- **Modello 2N – Parte I e II** → per unità delle categorie **D, E**  
- **Planimetrie catastali** → disegni delle singole unità immobiliari  
- **Elaborato planimetrico** → schema generale del fabbricato  
  (necessario se ci sono almeno due unità)

---

## Dettaglio: Modello 1N

### Ambito di utilizzo

Il **modello 1N parte I** è usato **solo per le categorie catastali ordinarie**:  
A, B, C

Viene utilizzato:
- sia per **nuove dichiarazioni (Accatastamenti)**  
- sia per **variazioni**  
ma **non** per le categorie D ed E (che usano il modello 2N)

---

### Relazione tra Parte I e Parte II

Anche se nel Docfa ci sono più unità immobiliari:

- Il **modello 1N parte I è uno solo** per tutto il **fabbricato**  
- Il **modello 1N parte II è uno per ciascuna unità** (appartamento, negozio, box, ecc.)

**In pratica:**

| Modello | Riferimento | Contenuto |
|----------|--------------|-----------|
| 1N Parte I | Fabbricato | Dati generali: indirizzo, strutture, impianti, ecc. |
| 1N Parte II | Unità immobiliare | Dati specifici: superficie, vani, pertinenze, ecc. |

---


In [13]:
import os
import xml.etree.ElementTree as ET

xml_folder = "../../../dati_AI_per_Challenge/Docfa/Docfa_anonimizzati_2013_2024-12-31"

tags_per_file = {}

# Due set distinti per i tipi di documenti
accatastamenti = set()
variazioni = set()
non_classificati = set()

for filename in os.listdir(xml_folder):
    if filename.lower().endswith(".xml"):
        path = os.path.join(xml_folder, filename)
        try:
            tree = ET.parse(path)
            root = tree.getroot()

            tags_and_attrs = set()
            for elem in root.iter():
                tags_and_attrs.add(elem.tag)
                for attr in elem.attrib.keys():
                    tags_and_attrs.add(f"{elem.tag}:{attr}")

            tags_per_file[filename] = tags_and_attrs

            # Identifica il tipo di documento
            tags_lower = {t.lower() for t in tags_and_attrs}
            if "accatastamento" in tags_lower:
                accatastamenti.add(filename)
            elif "variazione" in tags_lower:
                variazioni.add(filename)
            else:
                non_classificati.add(filename)

        except Exception as e:
            print(f"Error while reading {filename}: {e}")

# --- Calcolo tag comuni per accatastamenti e variazioni ---

def common_tags_for(files_subset):
    """Restituisce l'intersezione dei tag per un sottoinsieme di file."""
    if not files_subset:
        return set()
    sets = [tags_per_file[f] for f in files_subset if f in tags_per_file]
    return set.intersection(*sets) if sets else set()

# Tag comuni specifici per tipo documento
common_accatastamento = common_tags_for(accatastamenti)
common_variazione = common_tags_for(variazioni)

# --- Analisi complessiva ---
all_fields = set().union(*tags_per_file.values())
common_fields_all = set.intersection(*tags_per_file.values()) if tags_per_file else set()

print(f"Total files: {len(tags_per_file)}")
print(f"Accatastamenti: {len(accatastamenti)} file")
print(f"Variazioni: {len(variazioni)} file")
print(f"Non classificati: {len(non_classificati)} file")

print("\n--- TAG COMUNI PER TIPO DI DOCUMENTO ---")
print(f"Tag comuni negli ACCATASTAMENTI ({len(common_accatastamento)}):")
print(sorted(common_accatastamento))

print(f"\nTag comuni nelle VARIAZIONI ({len(common_variazione)}):")
print(sorted(common_variazione))


Total files: 12989
Accatastamenti: 1443 file
Variazioni: 11546 file
Non classificati: 0 file

--- TAG COMUNI PER TIPO DI DOCUMENTO ---
Tag comuni negli ACCATASTAMENTI (30):
['Accatastamento', 'Accatastamento:comuneCatastale', 'Accatastamento:descrizione', 'Accatastamento:lingua', 'Accatastamento:provincia', 'Accatastamento:tipoCausale', 'Accatastamento:{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation', 'Classamento', 'Classamento:categoriaImmobiliare', 'Classamento:sottoCategoriaImmobiliare', 'DatiTecnico', 'Ditta', 'Ditta:tipo', 'ElencoDitte', 'ElencoIdentificativiCatastaliPM', 'ElencoIndirizzi', 'ElencoPiani', 'ElencoUI', 'IdentificativoCatastalePM', 'IdentificativoCatastalePM:comuneCatastale', 'IdentificativoCatastalePM:foglio', 'IdentificativoCatastalePM:numeratore', 'Indirizzo', 'Indirizzo:codiceVia', 'Indirizzo:indirizzoIT', 'Piano', 'Piano:numeroPiano', 'RiferimentiNormativi', 'UICostituzione', 'UICostituzione:dataUsufruibilita']

Tag comuni nelle VARIAZIONI 

In [14]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

xml_folder = "../../../dati_AI_per_Challenge/Docfa/Docfa_anonimizzati_2013_2024-12-31"

records = []

for filename in os.listdir(xml_folder):
    if not filename.lower().endswith(".xml"):
        continue

    path = os.path.join(xml_folder, filename)
    try:
        tree = ET.parse(path)
        root = tree.getroot()

        # raccoglie tutti i tag (e i nomi degli attributi) in minuscolo
        tags_and_attrs = set()
        for elem in root.iter():
            tags_and_attrs.add(elem.tag.lower())
            for attr in elem.attrib.keys():
                tags_and_attrs.add(f"{elem.tag.lower()}:{attr.lower()}")

        # determina il tipo di documento
        if "accatastamento" in tags_and_attrs:
            tipo_docfa = "Accatastamento"
        elif "variazione" in tags_and_attrs:
            tipo_docfa = "Variazione"
        else:
            tipo_docfa = "Sconosciuto"

        records.append({"file": filename, "tipo_docfa": tipo_docfa})

    except Exception as e:
        print(f"Errore in {filename}: {e}")

# crea il dataframe finale
df = pd.DataFrame(records)

print(df.head())
print(df["tipo_docfa"].value_counts())


                                                file      tipo_docfa
0  0_MUT_1432792_777926_NCV_334_50_115_2_1_xml_an...      Variazione
1  0_MUT_1390596_559699_NCV_320_406_7104_0_25_xml...      Variazione
2  0_MUT_1365975_470972_NCA_243_285_1404_0_0_xml_...  Accatastamento
3  0_MUT_1488004_1156465_NCV_169_222_14_3_1_xml_a...      Variazione
4  0_MUT_1417082_677375_NCV_56_437_187_0_4_xml_an...      Variazione
tipo_docfa
Variazione        11546
Accatastamento     1443
Name: count, dtype: int64


In [16]:
"""
Organize all the fields of a specific xml file into a python dict.
"""

import xml.etree.ElementTree as ET

xml_file = "../../../dati_AI_per_Challenge/Docfa/Docfa_anonimizzati_2013_2024-12-31/0_MUT_1489992_1173741_NCV_41_17_144_0_5_xml_anonymus.xml"

tree = ET.parse(xml_file)
root = tree.getroot()

xml_dict = {}

for elem in root.iter():
    if elem.attrib:
        xml_dict[elem.tag] = elem.attrib

for k, v in xml_dict.items():
    print(f"{k}: {v}")

Variazione: {'comuneCatastale': '17', 'hashAllegatiPratica': 'yBWn24fOtx', 'provincia': 'TN', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'Docfa_v4.2.xsd', 'lingua': 'IT', 'descrizione': "CAMBIO DESTINAZIONE D'USO", 'variazioneDiDestinazione': 'S', 'variazioneDiDestinazioneDaDescr': 'UFFICIO', 'variazioneDiDestinazioneADescr': 'ABITAZIONE', 'dataEffettuazioneVariazioni': '2024-10-09', 'variazionePlanimetricaFrazionamento': 'S', 'variazionePlanimetricaDiversaDistribSpaziInterni': 'S'}
A_RiferimentiCatastali: {'esistonoUINonInserite': 'true'}
C1_ElementiDescrittivi: {'posizRispettoFabbCircostanti': 'I', 'complessoImmobiliare': 'true', 'fabbricatiNum': '2'}
C2_DestinazioneNumeroUI: {'abitazioniNum': '1'}
C3_ScaleAccesso: {'accessoTipo': 'C', 'scaleNum': '1'}
C4_CaratteristicheDimensionali: {'pianiFuoriTerraNum': '2'}
C6_Cortile: {'cortileInterno': 'true'}
D2_DestinazioneSpaziScoperti: {'verdeMq': '1162'}
E_TipoAffaccioPrevalente: {'stradaLargaFino10M01': 'S', '

## Leggere Catasto_geometrico

Funzione per leggere i file dentro `.shp` dentro la cartella selezionata.
L'idea dietro questo è che i punti trovati dentro i `.shp` file hanno le cordinate geografiche, quindi si potrebbe cercare un modo per collegarli con le unita immobiliare.

In [17]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

def process_shapefiles_in_directory(folder_path, save_csv=False):
    """
    Reads all .shp files in the given folder and creates 4 DataFrames for each:
     - df_attributes
     - df_geometry
     - df_projection
     - df_metadata
    Optionally saves them to CSV.
    """
    folder = Path(folder_path)
    shp_files = list(folder.glob("*.shp"))

    if not shp_files:
        print(f"No .shp files found in {folder}")
        return

    print(f"Found {len(shp_files)} shapefiles in {folder}\n")

    results = {}

    for shp_path in shp_files:
        print(f"🔹 Processing: {shp_path.name}")
        try:
            gdf = gpd.read_file(shp_path)

            # === 1. Attribute table ===
            df_attributes = pd.DataFrame(gdf.drop(columns="geometry"))

            # === 2. Geometry ===
            df_geometry = gdf[["geometry"]].copy()

            # === 3. Projection info ===
            crs = gdf.crs.to_string() if gdf.crs else None
            df_projection = pd.DataFrame([{"CRS": crs}])

            # === 4. Metadata ===
            df_metadata = pd.DataFrame({
                "Column": gdf.columns,
                "Type": [str(gdf[col].dtype) for col in gdf.columns],
                "Has Nulls": [gdf[col].isnull().any() for col in gdf.columns]
            })

            # Store results in dict
            results[shp_path.stem] = {
                "attributes": df_attributes,
                "geometry": df_geometry,
                "projection": df_projection,
                "metadata": df_metadata
            }

            # === Optional: save CSV files ===
            if save_csv:
                prefix = folder / shp_path.stem
                df_attributes.to_csv(f"{prefix}_attributes.csv", index=False)
                df_geometry.to_csv(f"{prefix}_geometry.csv", index=False)
                df_projection.to_csv(f"{prefix}_projection.csv", index=False)
                df_metadata.to_csv(f"{prefix}_metadata.csv", index=False)
                print(f"✅ Saved CSVs for {shp_path.name}")

            print(f"   CRS: {crs}")
            print(f"   Columns: {list(gdf.columns)}\n")

        except Exception as e:
            print(f"❌ Error processing {shp_path.name}: {e}\n")

    print("All shapefiles processed successfully ✅")
    return results


# === Example usage ===
if __name__ == "__main__":
    folder_path = "../../../dati_AI_per_Challenge/catasto_geometrico/406_shp"  # replace with your folder path
    all_results = process_shapefiles_in_directory(folder_path, save_csv=True)

Found 13 shapefiles in ../../../dati_AI_per_Challenge/catasto_geometrico/406_shp

🔹 Processing: 406_grid.shp
✅ Saved CSVs for 406_grid.shp
   CRS: EPSG:25832
   Columns: ['FM_CODE', 'FM_NUMFOG', 'FM_CCAT', 'geometry']

🔹 Processing: 406_vl_right_poly.shp
✅ Saved CSVs for 406_vl_right_poly.shp
   CRS: EPSG:25832
   Columns: ['RP_TYPE', 'RP_NFR', 'RP_CODE', 'RP_CCAT', 'RP_PUB', 'geometry']

🔹 Processing: 406_vl_boundary.shp
✅ Saved CSVs for 406_vl_boundary.shp
   CRS: EPSG:25832
   Columns: ['BD_PZ', 'BD_PUB', 'BD_PUB_IT', 'BD_PUB_DE', 'BD_CCAT', 'BD_NAME_IT', 'BD_NAME_DE', 'BD_ISTAT', 'BD_ADM_IT', 'BD_ADM_DE', 'BD_SUMPART', 'geometry']

🔹 Processing: 406_vl_local_name.shp
✅ Saved CSVs for 406_vl_local_name.shp
   CRS: EPSG:25832
   Columns: ['LN_CCAT', 'LN_LANG', 'LN_TYPE', 'LN_TEXT', 'LN_ANGLE', 'geometry']

🔹 Processing: 406_vl_boundary_line.shp
✅ Saved CSVs for 406_vl_boundary_line.shp
   CRS: EPSG:25832
   Columns: ['BL_CCAT', 'BL_ISTAT', 'BL_TYPE_DE', 'BL_TYPE_IT', 'BL_TYPE_V', 'ge

/Users/kuba/AIChallenge/virt_env_notebook/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: Value 'VL' of field 406_vl_local_name.LN_TYPE parsed incompletely to integer 0.
  return ogr_read(


✅ Saved CSVs for 406_vl_uniqueparcel_poly.shp
   CRS: EPSG:25832
   Columns: ['PU_TYPE', 'PU_CODE', 'PU_CCAT', 'PU_ISTAT', 'PU_NUM', 'PU_DENOM', 'PU_AREA', 'geometry']

🔹 Processing: 406_vl_symbols.shp
✅ Saved CSVs for 406_vl_symbols.shp
   CRS: EPSG:25832
   Columns: ['SM_TYPE', 'SM_ANGLE', 'SM_ANG_90', 'SM_NFR', 'SM_CCAT', 'SM_LEVEL', 'SM_PUB', 'geometry']

🔹 Processing: 406_vl_dress_line.shp
✅ Saved CSVs for 406_vl_dress_line.shp
   CRS: EPSG:25832
   Columns: ['DL_TYPE', 'DL_NFR', 'DL_CCAT', 'DL_COLOR', 'DL_PUB', 'geometry']

🔹 Processing: 406_vl_parcel_poly.shp
✅ Saved CSVs for 406_vl_parcel_poly.shp
   CRS: EPSG:25832
   Columns: ['PT_TYPE', 'PT_CODE', 'PT_CCAT', 'PT_FABB', 'PT_PUB', 'geometry']

🔹 Processing: 406_vl_parcel_line.shp
✅ Saved CSVs for 406_vl_parcel_line.shp
   CRS: EPSG:25832
   Columns: ['PL_TYPE', 'PL_NFR', 'PL_CCAT', 'PL_COLOR', 'PL_PUB', 'geometry']

🔹 Processing: 406_fix_points.shp
✅ Saved CSVs for 406_fix_points.shp
   CRS: EPSG:25832
   Columns: ['FP_CCAT', 

In [18]:
## ESEMPIO
directory="../../../dati_AI_per_Challenge/catasto_geometrico/406_shp" 
r=process_shapefiles_in_directory(directory)
for k,v in r.items():
  print(k)
  display(v["attributes"].head(5))

Found 13 shapefiles in ../../../dati_AI_per_Challenge/catasto_geometrico/406_shp

🔹 Processing: 406_grid.shp
   CRS: EPSG:25832
   Columns: ['FM_CODE', 'FM_NUMFOG', 'FM_CCAT', 'geometry']

🔹 Processing: 406_vl_right_poly.shp
   CRS: EPSG:25832
   Columns: ['RP_TYPE', 'RP_NFR', 'RP_CODE', 'RP_CCAT', 'RP_PUB', 'geometry']

🔹 Processing: 406_vl_boundary.shp
   CRS: EPSG:25832
   Columns: ['BD_PZ', 'BD_PUB', 'BD_PUB_IT', 'BD_PUB_DE', 'BD_CCAT', 'BD_NAME_IT', 'BD_NAME_DE', 'BD_ISTAT', 'BD_ADM_IT', 'BD_ADM_DE', 'BD_SUMPART', 'geometry']

🔹 Processing: 406_vl_local_name.shp
   CRS: EPSG:25832
   Columns: ['LN_CCAT', 'LN_LANG', 'LN_TYPE', 'LN_TEXT', 'LN_ANGLE', 'geometry']

🔹 Processing: 406_vl_boundary_line.shp
   CRS: EPSG:25832
   Columns: ['BL_CCAT', 'BL_ISTAT', 'BL_TYPE_DE', 'BL_TYPE_IT', 'BL_TYPE_V', 'geometry']

🔹 Processing: 406_vl_right_line.shp
   CRS: EPSG:25832
   Columns: ['DL_ISTAT', 'DL_CCAT', 'DL_NFR', 'DL_TYPE_V', 'DL_TYPE_DE', 'DL_TYPE_IT', 'DL_QUAL_V', 'DL_QUAL_DE', 'DL_QUAL

/Users/kuba/AIChallenge/virt_env_notebook/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: Value 'VL' of field 406_vl_local_name.LN_TYPE parsed incompletely to integer 0.
  return ogr_read(


   CRS: EPSG:25832
   Columns: ['SM_TYPE', 'SM_ANGLE', 'SM_ANG_90', 'SM_NFR', 'SM_CCAT', 'SM_LEVEL', 'SM_PUB', 'geometry']

🔹 Processing: 406_vl_dress_line.shp
   CRS: EPSG:25832
   Columns: ['DL_TYPE', 'DL_NFR', 'DL_CCAT', 'DL_COLOR', 'DL_PUB', 'geometry']

🔹 Processing: 406_vl_parcel_poly.shp
   CRS: EPSG:25832
   Columns: ['PT_TYPE', 'PT_CODE', 'PT_CCAT', 'PT_FABB', 'PT_PUB', 'geometry']

🔹 Processing: 406_vl_parcel_line.shp
   CRS: EPSG:25832
   Columns: ['PL_TYPE', 'PL_NFR', 'PL_CCAT', 'PL_COLOR', 'PL_PUB', 'geometry']

🔹 Processing: 406_fix_points.shp
   CRS: EPSG:25832
   Columns: ['FP_CCAT', 'FP_CODE', 'FP_CCODE', 'FP_ATTEND', 'FP_MATER', 'FP_QUALPLA', 'FP_QUALORT', 'FP_Q_ORTO', 'FP_Q_ELLI', 'FP_GEOG_FI', 'FP_GEOG_LA', 'FP_UTM_E', 'FP_UTM_N', 'geometry']

🔹 Processing: 406_vl_dress_poly.shp
   CRS: EPSG:25832
   Columns: ['DP_TYPE', 'DP_NFR', 'DP_CCAT', 'DP_COLOR', 'DP_PUB', 'geometry']

All shapefiles processed successfully ✅
406_grid


,FM_CODE,FM_NUMFOG,FM_CCAT
0,060052E,1,406
1,060063A,2,406
2,060052J,3,406
3,060063F,4,406
4,060052P,5,406


406_vl_right_poly


...

,PL_TYPE,PL_NFR,PL_CCAT,PL_COLOR,PL_PUB
0,1,2021-05419-001,406,R,YES
1,1,2021-05414-001,406,R,YES
2,1,2021-05414-001,406,R,YES
3,1,2015-06002-001,406,N,YES
4,1,2015-06002-001,406,N,YES


406_fix_points


,FP_CCAT,FP_CODE,FP_CCODE,FP_ATTEND,FP_MATER,FP_QUALPLA,FP_QUALORT,FP_Q_ORTO,FP_Q_ELLI,FP_GEOG_FI,FP_GEOG_LA,FP_UTM_E,FP_UTM_N
0,406,102,406102,4,NaN,NaN,NaN,296.60,346.05,46.068189,11.134814,665099.01,5103839.33
1,406,119,406119,4,NaN,NaN,NaN,197.55,247.02,46.070524,11.111189,663265.10,5104050.00
2,406,150,406150,4,NaN,NaN,NaN,194.12,243.57,46.067539,11.121242,664051.35,5103739.04
3,406,217,406217,5,NaN,NaN,NaN,0.00,0.00,46.072367,11.124925,664321.85,5104283.05
4,406,198,406198,4,NaN,NaN,NaN,206.15,255.60,46.054969,11.129419,664721.14,5102359.39


406_vl_dress_poly


,DP_TYPE,DP_NFR,DP_CCAT,DP_COLOR,DP_PUB
0,32,2013-00860-001,406,R,YES
1,32,2018-00887-001,406,R,YES
2,28,2019-00659-001,406,R,YES
3,32,2013-00413-001,406,R,YES
